<center>
<p><img src="https://www.gob.mx/cms/uploads/image/file/179499/outstanding_quienes-somos.jpg" width="300">
</p>



# Curso *Machine Learning con uso de pandas, scikit learn y libretas jupyter*

# Analisis demanda Gerncia Oriental 


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="80">
</p>
</center>


In [ ]:
!pip install skforecast
!pip install -U openpyxl

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 7]

In [ ]:
url = "https://github.com/juliowaissman/curso-ml-cenace/raw/main/datos/historico_demanda_temp.xlsx"

df_demanda = pd.read_excel(url, sheet_name=0)
df_temperatura = pd.read_excel(url, sheet_name=1)

# Elimina los comentarios si quieres ver la información de los dataframes
print(df_demanda.info())
print(df_temperatura.info())

In [83]:
def estira(df, name):
  df = df.melt(
    id_vars=['FECHA'], 
    value_vars=list(range(1, 25)),
    var_name='Hora',
    value_name=name
  )
  df.index = df['FECHA'] + pd.to_timedelta((df.Hora.values - 1).tolist(), unit='h')
  df.sort_index(inplace=True)
  df.drop(columns=['FECHA', 'Hora'], inplace=True)
  return df

df_demanda.rename(columns={'Unnamed: 1': 'FECHA'}, inplace=True)
df_demanda = estira(df_demanda, 'Demanda')
df_temperatura = estira(df_temperatura, 'Temperatura')

df = df_demanda.join(df_temperatura.Temperatura)

In [ ]:
df

In [ ]:
def elimina_no_numericos(df, var, verbose=True):
  "Elimina valores no numericos de una serie y la convierte a dtype numerico"

  def no_numerico(serie):
    "Aplica una función que verifica si el elemento no es float o int"
    return serie.apply(
      lambda x: type(x) != type(3.14) and type(x) != type(1)
    )

  if verbose:
    print("**Eliminando valores no numéricos**")
    print(df.loc[no_numerico(df[var]), var])
  df.loc[no_numerico(df[var]), var] = np.NaN
  df[var] = pd.to_numeric(df[var])
  return df

df = elimina_no_numericos(df, 'Demanda')
df = elimina_no_numericos(df, 'Temperatura')

df.Demanda.fillna(method='pad', inplace=True)
df.Temperatura.fillna(method='pad', inplace=True)

In [ ]:
df[(df.index.month == 8) &
   (df.index.year == 2021)].Temperatura.plot(title="Temperatura agosto 2021")

In [ ]:
df = df[df.index < pd.to_datetime('18-8-2021', format="%d-%m-%Y")]

df[(df.index.month == 8) &
   (df.index.year == 2021)].Temperatura.plot(title='Temperatura agosto 2021')

In [ ]:
df.Demanda.plot(style='b', title="Demanda histórica")

In [ ]:
año, mes = 2019, 4
df[(df.index.year == año) & 
   (df.index.month == mes)].Demanda.plot(title=f"Demanda de energia, año {año}, mes {mes}")

Vamos a modificar los datos extremos debido al cambio de horario

In [ ]:
print("**Estas son las fechas que vamos a cambiar")
print(df.loc[(df.Demanda > 8000) | (df.Demanda < 500), 'Demanda'])
df.loc[(df.Demanda > 8000) | (df.Demanda < 500), 'Demanda'] = np.NaN
df.Demanda.fillna(method='pad', inplace=True)

df.Demanda.plot(style='b', title="Demanda histórica con corrección por cambio de horario")

In [101]:
# Vamos a asegurarnos que hay valor para todas las horas
df = df.asfreq('H', method='pad')

In [ ]:
df.groupby(df.index.weekday).agg(
    {
        'Demanda': ['min', 'max', 'mean', 'median', 'std', 'mad']
    }
)

In [ ]:
df.groupby(df.index.month).agg(
    {
        'Demanda': ['min', 'max', 'mean'],
        'Temperatura': ['min', 'max', 'mean']     
    }
)

In [ ]:
def barras(df, temporalidad, variable):
    tempo = {
        'año': df.index.year, 
        'mes': df.index.month,
        'dia': df.index.weekday,
        'hora': df.index.hour
    }
    ax = df[[variable]].groupby(tempo[temporalidad]).boxplot(subplots=False, rot=90)
    ax.set_title(f'Evolción de la {variable} por {temporalidad}')
    plt.show()

barras(df[df.index.year==2019], 'mes', 'Demanda')
barras(df[df.index.year==2019], 'mes', 'Temperatura')

## Forecasting


### 1. Encontrar los mejores parámetros 

In [170]:
df_train = df[df.index.year < 2020]
df_val = df[df.index.year == 2021]

In [ ]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster

forecaster = ForecasterAutoreg(
    regressor= Ridge(normalize=True),
    lags= 24 # Este valor será remplazado en el grid search
)

# Hiperparámetros del regresor
param_grid = {'alpha': np.logspace(-5, -1, 5)}

# Lags utilizados como predictores
lags_grid = [
  24, 
  [1, 2, 3, 24-1, 24, 24+1, 48-1, 48, 48+1],
  [1, 2, 3, 24-1, 24, 24+1, 48-1, 48, 48+1, 
   24 * 7 - 3, 24 * 7 - 2, 24 * 7 - 1, 24 * 7, 24 * 7 + 1, 24 * 7 + 2, 24 * 7 + 3]
]

resultados_grid = grid_search_forecaster(
    forecaster= forecaster,
    y= df_train.Demanda,
    param_grid= param_grid,
    lags_grid= lags_grid,
    steps= 36,
    metric= 'mean_absolute_error',
    method= 'backtesting',
    initial_train_size = len(df_train.Demanda[df_train.index.year < 2019]),
    return_best= True,
    verbose= False
)

In [ ]:
resultados_grid

In [ ]:
forecaster

Para probar nuestro forcaster, tenemos que usar la función para estimar las 24 horas del día siguiente pero 12 horas antes.

In [123]:
def backtest_predict_next_24h(
    forecaster, y, hour_init_prediction, exog=None, verbose=False):
    '''
    Backtest ForecasterAutoreg object when predicting 24 hours of day D+1
    statring at specific hour of day D.
    
    Parameters
    ----------
    forecaster : ForecasterAutoreg 
        ForecasterAutoreg object already trained.
        
    y : pd.Series with datetime index sorted
        Test time series values. 
        
    exog : pd.Series or pd.Dataframe with datetime index sorted
        Test values of exogen variable. 
    
    hour_init_prediction: int 
        Hour of day D to start predictions of day D+1.


    Returns 
    -------
    predictions: pd.Series
        Value of predictions.

    '''
    
    y = y.sort_index()
    if exog is not None:
        exog = exog.sort_index()
        
    dummy_steps = 24 - (hour_init_prediction + 1)
    steps = dummy_steps + 24
    
    # First position of `hour_init_prediction` in the series where there is enough
    # previous window to calculate lags.
    for datetime in y.index[y.index.hour == hour_init_prediction]:
        if len(y[:datetime]) >= len(forecaster.last_window):
            datetime_init_backtest = datetime
            print(f"Backtesting starts at day: {datetime_init_backtest}")
            break
    
    days_backtest = np.unique(y[datetime_init_backtest:].index.date)
    days_backtest = pd.to_datetime(days_backtest)
    days_backtest = days_backtest[1:]
    print(f"Days predicted in the backtesting: {days_backtest.strftime('%Y-%m-%d').values}")
    print('')
    backtest_predictions = []
    
    for i, day in enumerate(days_backtest):        
        # Start and end of the last window used to create the lags
        end_window = (day - pd.Timedelta(1, unit='day')).replace(hour=hour_init_prediction)
        start_window = end_window - pd.Timedelta(forecaster.max_lag, unit='hour')
        last_window = y.loc[start_window:end_window]
               
        if exog is None:
            if verbose:
                print(f"Forecasting day {day.strftime('%Y-%m-%d')}")
                print(f"Using window from {start_window} to {end_window}")
                
            pred = forecaster.predict(steps=steps, last_window=last_window)
            
        else:
            start_exog_window = end_window + pd.Timedelta(1, unit='hour')
            end_exog_window   = end_window + pd.Timedelta(steps, unit='hour')
            exog_window = exog.loc[start_exog_window:end_exog_window]
            exog_window = exog_window.to_numpy()
            
            if verbose:
                print(f"Forecasting day {day.strftime('%Y-%m-%d')}")
                print(f"    Using window from {start_window} to {end_window}")
                print(f"    Using exogen variable from {start_exog_window} to {end_exog_window}")
            
            pred = forecaster.predict(steps=steps, last_window=last_window, exog=exog_window)

        # Only store predictions of day D+1
        pred = pred[dummy_steps:]
        backtest_predictions.append(pred)
    
    backtest_predictions = np.concatenate(backtest_predictions)
    # Add datetime index
    backtest_predictions = pd.Series(
                             data  = backtest_predictions,
                             index = pd.date_range(
                                        start = days_backtest[0],
                                        end   = days_backtest[-1].replace(hour=23),
                                        freq  = 'h'
                                    )
                           )
    
    return backtest_predictions

In [124]:
def mean_absolute_percent_error(y_true, y_pred):
  error = np.abs((y_true - y_pred) / y_true)
  return error.mean()

In [ ]:
y_pred = backtest_predict_next_24h(
  forecaster= forecaster,
  y= df_val.Demanda,
  hour_init_prediction= 11,
  verbose= False
)
y_true = df_val.loc[predicciones.index, 'Demanda']

error_MAE = mean_absolute_error( y_true=y_true, y_pred=y_pred)
error_MAPE = mean_absolute_percent_error(y_true=y_true, y_pred=y_pred)

print(f"Error de backtest (MAE): {error_MAE}")
print(f"Error de backtest (MAPE): {error_MAPE}")

In [136]:
def plot_semana_demanda(y_true, y_pred, semana):
  fig, ax = plt.subplots()
  y_true[y_true.index.week == semana]\
      .plot(ax=ax, lw=2, label='validación')
  y_pred[y_pred.index.week == semana]\
      .plot(ax=ax, lw=2, label='predicción')
  ax.set_title('Predicción vs demanda real')
  ax.legend()
  plt.show()

In [ ]:
semana = 2
plot_semana_demanda(y_true, y_pred, semana)

In [145]:
def por_dia(y_true, y_pred, umbral, grafica=True):
  APE = abs((y_true - y_pred) / y_pred)
  diario = APE.groupby(APE.index.day_of_year).mean()
  inaceptable = diario[diario > umbral]
  if grafica:
    y_true_diaria = y_true.groupby(y_true.index.day_of_year).mean()
    y_pred_diaria = y_pred.groupby(y_pred.index.day_of_year).mean()
    plt.plot(y_true_diaria.index, y_true_diaria.values, 'b')
    plt.plot(y_pred_diaria.index, y_pred_diaria.values, 'c')
    plt.plot(
        inaceptable.index, 
        y_true_diaria[y_true_diaria.index.isin(inaceptable.index)],
        '*r'
    ) 
    plt.show()
  return diario, inaceptable

def por_hora(y_true, y_pred, umbral, grafica=True):
  APE = abs((y_true - y_pred) / y_pred)
  inaceptable = APE[APE > umbral]
  if grafica:
    plt.plot(y_true.index, y_true.values, 'b')
    plt.plot(y_pred.index, y_pred.values, 'c')
    plt.plot(
        inaceptable.index, 
        y_true[y_true.index.isin(inaceptable.index)],
        '*r'
    ) 
    plt.show()
  return APE, inaceptable

In [ ]:
mape_diario, inaceptable = por_dia(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} días inaceptables")

ape, inaceptable = por_hora(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} horas inaceptables")

In [ ]:
semana = 2
ape, inaceptable = por_hora(y_true[y_true.index.weekofyear==semana], 
                            y_pred[y_pred.index.weekofyear==semana], 0.05)

## Agregando temperatura


In [ ]:
forecasterT = ForecasterAutoreg(
    regressor= Ridge(alpha=0.0001, normalize=True),
    lags= [1, 2, 3, 4, 23, 24, 25, 47, 48, 49, 165, 166, 167, 168, 169, 170, 171]
)
forecasterT.fit(y=df_train.Demanda, exog=df_train.Temperatura)

y_pred = backtest_predict_next_24h(
  forecaster= forecasterT,
  y= df_val.Demanda,
  exog=df_val.Temperatura,
  hour_init_prediction= 11,
  verbose= False
)
y_true = df_val.loc[y_pred.index, 'Demanda']

error_MAE = mean_absolute_error( y_true=y_true, y_pred=y_pred)
error_MAPE = mean_absolute_percent_error(y_true=y_true, y_pred=y_pred)

print(f"Error de backtest (MAE): {error_MAE}")
print(f"Error de backtest (MAPE): {error_MAPE}")

In [ ]:
mape_diario, inaceptable = por_dia(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} días inaceptables")

ape, inaceptable = por_hora(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} horas inaceptables")

# Agregando otras variables exógenas

In [ ]:
import holidays

festivos = list(
    holidays.MEX(years=[
        2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 
        2017, 2018, 2019, 2020, 2021]).keys()
)
df['Holiday'] = 1
df.Holiday.where(df.index.isin(festivos), 0, inplace=True)

df['Semana'] = df.index.weekofyear
df['Hora'] = df.index.hour
df['Dia'] = df.index.weekday
df['Mes'] = df.index.month


df_train = df[df.index.year < 2020]
df_val = df[df.index.year == 2021]

In [ ]:
exogenas = ['Temperatura', 'Dia']

forecasterX = ForecasterAutoreg(
    regressor= Ridge(alpha=0.0001, normalize=True),
    lags= [1, 2, 3, 4, 23, 24, 25, 47, 48, 49, 165, 166, 167, 168, 169, 170, 171]
)
forecasterX.fit(y=df_train.Demanda, exog=df_train[exogenas].values)

y_pred = backtest_predict_next_24h(
  forecaster= forecasterX,
  y= df_val.Demanda,
  exog=df_val[exogenas],
  hour_init_prediction= 11,
  verbose= False
)
y_true = df_val.loc[y_pred.index, 'Demanda']

error_MAE = mean_absolute_error( y_true=y_true, y_pred=y_pred)
error_MAPE = mean_absolute_percent_error(y_true=y_true, y_pred=y_pred)

print(f"Error de backtest (MAE): {error_MAE}")
print(f"Error de backtest (MAPE): {error_MAPE}")

In [ ]:
mape_diario, inaceptable = por_dia(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} días inaceptables")

ape, inaceptable = por_hora(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} horas inaceptables")

## Usando variables *dummies*

In [206]:
# One hot encoding de las variables mes, hora y dia
df_dummies=pd.get_dummies(df, columns=['Hora', 'Dia', 'Mes', 'Semana'])
df_train = df_dummies[df_dummies.index.year < 2020]
df_val = df_dummies[df_dummies.index.year > 2020]

In [ ]:
exogenas = [column for column in df_dummies.columns 
            if column.startswith(('Dia', 'Temperatura'))]

forecasterX = ForecasterAutoreg(
    regressor= Ridge(alpha=0.0001, normalize=True),
    lags= [1, 2, 3, 4, 23, 24, 25, 47, 48, 49, 165, 166, 167, 168, 169, 170, 171]
)
forecasterX.fit(y=df_train.Demanda, exog=df_train[exogenas].values)

y_pred = backtest_predict_next_24h(
  forecaster= forecasterX,
  y= df_val.Demanda,
  exog=df_val[exogenas],
  hour_init_prediction= 11,
  verbose= False
)
y_true = df_val.loc[y_pred.index, 'Demanda']

error_MAE = mean_absolute_error( y_true=y_true, y_pred=y_pred)
error_MAPE = mean_absolute_percent_error(y_true=y_true, y_pred=y_pred)

print(f"Error de backtest (MAE): {error_MAE}")
print(f"Error de backtest (MAPE): {error_MAPE}")


In [ ]:
mape_diario, inaceptable = por_dia(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} días inaceptables")

ape, inaceptable = por_hora(y_true, y_pred, 0.05)
print(f"Hubo {len(inaceptable)} horas inaceptables")